1) Loading and cleaning the data

In [51]:
import pandas as pd
import numpy as np

In [88]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

#print('Before dropping Boroughs ='.join(df.shape))
df= df[df['Borough'] != 'Not assigned'].reset_index(drop= True)

df.groupby('Postal Code').count()>1

df[]

,Borough,Neighborhood
Postal Code,,
M1B,False,False
M1C,False,False
M1E,False,False
M1G,False,False
M1H,False,False
M1J,False,False
M1K,False,False
M1L,False,False
M1M,False,False


(103, 3)